<a href="https://colab.research.google.com/github/srijan0709/Rough-GPT/blob/main/RoughGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

In [4]:
!mkdir Ug_rulebook

In [7]:
loader = PyPDFDirectoryLoader("Ug_rulebook")
data = loader.load()

In [8]:
data

[Document(page_content='1  \n \n \n \n \n \n \nApplicable to the B.Tech., B.S., B.Des.,   \nDual Degree students admitted from the  \nAcademic Year 2007 - 2008 \n \n \n \n \n \n \n \nUpdated: May, 2023  ', metadata={'source': 'Ug_rulebook/ugrulebook.pdf', 'page': 0}),
 Document(page_content='2 Rules are classified into three separate categories as follows: (I) those which may be implemented \nwithin a department by DUGC/DPGC, (ii) those that require a decision at the level of A/Dean AP, or \nUGAPEC/PGAPEC, based on recommendations from the department bodies (iii) those that need to \nbe discussed in the Senate for a decision. \n \nTherefore, rules are colored with one of three colors.  \n1. The color green indicates that the final authority for rule is the Convener DUGC \n2. The color yellow, and underlined means that the final authority is Associate Dean (AP)/Dean \n(AP). \n3. The color yellow, without an underline means that the Convener, UGAPEC is the authority. \n4. The color blue 

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [10]:
docs = text_splitter.split_documents(data)

In [11]:
len(docs)

261

In [12]:
docs[260]

Document(page_content='SPI  : Semester Performance Index which is obtained by dividing the Se-\nmester Grade Points by the Semester Credits. \n \nSummer Term  : Approximately of 8 weeks duration from May to July. \n \nTranscript  : Consolidated statement of the Academic Performance of a student \nfor all the semesters completed. \n \nUGAPEC  : Under-Graduate Academic Performance Evaluation Committee \n \nUGPC  : Under-Graduate Programmes Committee \n \nURA  : Undergraduate Research Award. \n \n \n-------xxxx------', metadata={'source': 'Ug_rulebook/ugrulebook.pdf', 'page': 45})

In [13]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("I am Srijan")

In [15]:
len(query_result)

384

In [16]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '9ad66fd9-5adb-44ea-92a0-fa68f53c1291')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [17]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "roughgpt"

In [18]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [19]:
query = "What is the IDDDP program"

ans = docsearch.similarity_search(query, k=2)

In [20]:
ans

[Document(page_content='IDDDP, (with/without HONORS) which may differ depending on the parent academic unit of \nthe incoming student to facilitate introduction considering (a). \ne) The DUGC/DPGC of the admitting academic unit must prescribe beforehand and certify the \ncompletion of the HONORS requirement. An academic unit can admit at most 2 UG students in \neach DD/ M.Tech. specialization.  \nf) The selection and entry of all candidates in IDDDP will remain provisional till the successful'),
 Document(page_content='d) Recommendations from the faculty advisor/ guide \nNote that students must find out the exact guidelines from their respective departments.  \n14. GUIDELINES FOR INTERDISCIPLINARY DUAL DEGREE PROGRAMME (IDDDP) (Ref: 240th   Sen-\nate Meeting)  \nA. Minimum Eligibility Criteria:  \na) Undergraduate (UG) students admitted to B.S., B.Tech. and DD (B.Tech.+M.Tech.) programmes \ncan apply for IDDDP at the end of sixth semester.')]

In [21]:
from huggingface_hub import notebook_login
import torch


In [22]:
notebook_login()

In [23]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [24]:
!pip install accelerate
!pip install bitsandbytes




import bitsandbytes
# from bnb.nn import Linear8bitLt

import accelerate


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [25]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [26]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.15})

In [27]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS
instruction = """
{context}

Question: {question}
"""
template = B_INST + SYSTEM_PROMPT + instruction + E_INST
template

"[INST]<>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [28]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [41]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [30]:
result = qa_chain("What is the IDDDP program")



In [31]:
result['result']

'  Based on the provided context, the IDDDP program appears to be a program that offers admission to undergraduate students in various disciplines, including DD/M.Tech. specializations. The program seems to have different requirements and criteria for admission, depending on the parent academic unit of the incoming student.\nThe program also specifies that the DUGC/DPGC of the admitting academic unit must prescribe and certify the completion of the HONORS requirement before admitting students. Additionally, the program limits the number of students that an academic unit can admit in each DD/M.Tech. specialization to two.\nOverall, based on the provided context, the IDDDP program appears to be a program that facilitates admission to undergraduate students in various disciplines, with different requirements and criteria for admission depending on the parent academic unit.'

In [32]:
result1 = qa_chain("How to calculate SPI")

In [33]:
result1['result']

'  Based on the given context, the formula to calculate the Semester Performance Index (SPI) is:\nSPI = (C1g1 + C2g2 + C3g3 + C4g4 + C5g5) / (C1 + C2 + C3 + C4 + C5)\nWhere:\n* C1, C2, C3, C4, and C5 are the credits of the courses taken in the semester.\n* g1, g2, g3, g4, and g5 are the grade points earned in each of the courses.\n* The SPI is calculated to two decimal places.\n* The SPI for any semester will take into consideration the total number of credits and grade points earned in that semester.'

In [34]:
result2  = qa_chain("What is the role of a faculty advisor")

In [35]:
result2['result']

'  Based on the provided context, the role of a faculty advisor is:\ni. Guide the students about the rules and regulations governing the courses of study.\nii. Advise the students to register for courses as per the curriculum and their academic standing.'

In [36]:
result3  = qa_chain("what is the duration of DDP")
result3['result']

'  Based on the information provided in the context, the duration of Dual Degree Projects (DDPs) is fourteen months.'

In [37]:
result4 = qa_chain("who to consult before dropping any course")
result4['result']

'  Based on the provided context, the person to consult before dropping any course is:\ni. The Faculty Adviser\n\nYou can consult the Faculty Adviser for dropping a course or an entire semester due to health problems or other valid reasons beyond your control.'

In [38]:
result5 = qa_chain("who will evaluate my dual degree project")
result5['result']

'  Based on the context provided, it appears that the evaluation of the Dual Degree Project will be done by a committee. The exact composition of the committee is not specified in the provided text, but it is mentioned that the committee will evaluate the Final Stage of the Dual Degree Project. Therefore, the answer to your question is:\nThe committee evaluating your Dual Degree Project.'

In [39]:
result6 = qa_chain("what is the cpi requirements for branch change")
result6['result']

'  Based on the context provided, the CPI requirements for branch change are:\n(I) There is no student with either higher "Branch-Change-CPI" or same "Branch-Change-CPI" and higher "overall-CPI" who is currently being denied a change from any other branch X to branch B due to the following reasons:\n(II) There is no student with either higher "Branch-Change-CPI" or same "Branch-Change-CPI" and higher "overall-CPI" who is currently being denied a change from any other branch X to branch B.\nBased on the information provided, it seems that there are no specific CPI requirements mentioned for branch change. Therefore, I don\'t know the answer to your question.'

In [44]:
result7 = qa_chain("what is the minimum credit requirement for minor degree?")
result7['result']

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'  Based on the provided context, the minimum credit requirement for a minor degree is 30 credits.'

In [45]:
result8 = qa_chain("how many students take admission in IIT bombay each year?")
result8['result']

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'  Based on the information provided in the context, the number of students who take admission in IIT Bombay each year is:\n* 30 candidates for B.Des. Programme through UCEED\n* 300 candidates for M.Sc. and M.Sc. Ph.D. Dual Degree programmes\n* More than 1000 candidates for postgraduate programmes\n* Around 300 candidates for Ph.D. programmes\nSo, the total number of students who take admission in IIT Bombay each year is around 1330 (30 + 300 + 1000 + 300).'

In [48]:
result9 = qa_chain("how many students take admission in B.tech programmes at IIT Bombay?")
result9['result']

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'  Based on the information provided in the context, the answer to the question "How many students take admission in B.Tech programmes at IIT Bombay?" is:\nMore than 1000 students take admission in B.Tech programmes at IIT Bombay on an average annually through the Joint Entrance Examination (JEE).'